## Workflow

### 1. Setup and Authentication
- Import required libraries
- Load API key from environment variables
- Authenticate and fetch your Prolific researcher ID

In [ ]:
import json
import os
import requests
from google.colab import userdata
import yaml

import pandas as pd
import matplotlib.pyplot as plt

from prolific_helpers import (
    get_researcher_id,
    create_survey,
    create_study,
    publish_study,
    show_study_results,
    plot_survey_responses
)

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Use Colab secrets to add API token --> Tutorial: https://www.youtube.com/watch?v=LPa51KxqUAw
prolific_token = userdata.get('PROLIFIC_TOKEN')
prolific_workspace = userdata.get('PROLIFIC_WORKSPACE_ID')
prolific_project = userdata.get('PROLIFIC_PROJECT_ID')

In [3]:
# Set up headers for authenticated requests to the Prolific API
headers = {
    "Authorization": f"Token {prolific_token}",
    "Content-Type": "application/json",
}

In [ ]:
# Fetch your Prolific researcher ID
researcher_id = get_researcher_id(headers)

### 2. Survey Creation

In [ ]:
# Build survey question from config
question_text = config['survey']['question_text']
print(question_text)

In [ ]:
# Create survey using helper function
survey_id = create_survey(headers, researcher_id, config['survey'])
print(f"Survey created with ID: {survey_id}")

In [ ]:
# This cell is no longer needed - survey creation is handled by the helper function

In [ ]:
# This cell is no longer needed - survey creation is handled by the helper function

In [ ]:
# Survey ID is already created in the previous cell

### 3. Study Configuration
- Set study parameters (configurable):
  - Reward: $1 per participant
  - Participants: 20
  - Estimated time: 1 minute
- Generate completion codes and filters

In [ ]:
# Study configuration is now loaded from config.yaml
# Access it via: config['participants'], config['study'], config['device_compatibility']

### 4. Study Publication
- Create the study with all parameters
- Publish it to make it available to participants
- Monitor study status and participant recruitment

In [ ]:
# Completion code is now generated within the create_study helper function

In [ ]:
# Prepare study configuration from config.yaml
study_config = {
    "name": config['study']['name'],
    "description": config['study']['description'],
    "privacy_notice": config['study']['privacy_notice'],
    "reward": config['participants']['reward'],
    "participants": config['participants']['total_participants'],
    "estimated_time": config['participants']['estimated_time'],
    "max_time": config['participants']['max_time'],
    "device_compatibility": config['device_compatibility']
}

# Create study using helper function
study_id = create_study(headers, survey_id, study_config, prolific_project)
print(f"Study created with ID: {study_id}")

In [ ]:
# Study creation is now handled by the helper function above

In [ ]:
# Study ID is already created above

In [ ]:
# Publish study using helper function
status_code = publish_study(headers, study_id)
print(f"Study published with status code: {status_code}")

In [ ]:
# Status code is already printed above

# RESULTS
- Monitor participant submissions
- Export results in CSV format
- Load data into pandas DataFrame for analysis
- Calculate completion times and response statistics

In [ ]:
# show_study_results function is now imported from prolific_helpers.py

In [ ]:
# Fetch and display study results using helper function and configured timezone
df = show_study_results(study_id, headers, config['timezone'])

In [ ]:
df.head()

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Student status,Employment status,Long-term health condition/disability,Fluent languages,Sexual orientation,Highest education level completed,Degree subject,Work role,Submission approval rate,Do you think the new iPhone organge color would go well with a pink phone case?
0,6902c9dd0e1ffba0bb1453c3,62f975cbc3507ecf77180d00,AWAITING REVIEW,Not Applicable,2025-10-30T02:13:51.195000Z,2025-10-30T02:14:17.673000Z,NaN,2025-10-30T02:14:18.078000Z,27,NOCODE,...,DATA_EXPIRED,DATA_EXPIRED,No,English,DATA_EXPIRED,DATA_EXPIRED,DATA_EXPIRED,DATA_EXPIRED,100,"No, it would look bad!"
1,6902c9eb1ae8ed7baf7ce224,65c1bd40f77b5db9d93030b0,AWAITING REVIEW,Not Applicable,2025-10-30T02:14:04.742000Z,2025-10-30T02:14:24.405000Z,NaN,2025-10-30T02:14:24.844000Z,20,NOCODE,...,No,Full-Time,Yes,"German, Turkish, English",heterosexual,Graduate degree (MA/MSc/MPhil/other),"Engineering, manufacturing and construction",Manager,100,"No, it would look bad!"
2,6902ca122ee9f7189961d070,646e12b8cca43e4c726c45be,AWAITING REVIEW,Not Applicable,2025-10-30T02:14:46.600000Z,2025-10-30T02:16:17.788000Z,NaN,2025-10-30T02:16:18.247000Z,92,NOCODE,...,DATA_EXPIRED,DATA_EXPIRED,Yes,English,DATA_EXPIRED,DATA_EXPIRED,DATA_EXPIRED,DATA_EXPIRED,100,"Yes, it would look great!"
3,6902ca198d909200543039f8,669a7bf765ea1aa5d1389dd9,AWAITING REVIEW,Not Applicable,2025-10-30T02:14:52.816000Z,2025-10-30T02:15:22.682000Z,NaN,2025-10-30T02:15:23.166000Z,30,NOCODE,...,DATA_EXPIRED,DATA_EXPIRED,DATA_EXPIRED,"Japanese, English, Croatian, Portuguese",heterosexual,DATA_EXPIRED,Arts & Humanities,DATA_EXPIRED,100,"Yes, it would look great!"
4,6902ca38daf8f65bff9deaac,65f25f148039424910f7fb79,AWAITING REVIEW,Not Applicable,2025-10-30T02:15:23.893000Z,2025-10-30T02:15:36.342000Z,NaN,2025-10-30T02:15:36.779000Z,13,NOCODE,...,No,Full-Time,No,"Vietnamese, English",heterosexual,Undergraduate degree (BA/BSc/other),"Engineering, manufacturing and construction",Senior Manager,100,"Yes, it would look great!"


In [ ]:
# Plot survey responses using helper function
question_column = config['survey']['question_text']
fig, ax = plot_survey_responses(df, question_column)
plt.show()